In [ ]:
import tensorflow as tf
import math

# Softmax Regression

## Model Setting

In [ ]:
# input, answer
x = tf.placeholder(dtype = tf.float64, shape=[None, 28 * 28], name = 'input')
y_ = tf.placeholder(dtype = tf.float64, shape=[None, 10], name = 'correct')

# weight and bias
w = tf.Variable(tf.zeros([28 * 28, 10], dtype =tf.float64), name = 'weight')
b = tf.Variable(tf.zeros([10], dtype=tf.float64), name = 'bias', dtype=tf.float64)

# prediction
y = tf.nn.softmax(tf.matmul(x, w) + b)

# cross entropy
# sum of gradient for each training data. reduction_indices = [1]
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), 1)) 


## load Input

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## training

In [ ]:
learning_rate = 0.5
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

# training loop
for i in range(50):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_:batch[1]})


# Model Evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

sess.close()

----

# Fully connected feedforward network

## Model Setting

In [ ]:
x = tf.placeholder(dtype = tf.float64, shape = [None, 28*28], name = 'input')
y_ = tf.placeholder(dtype = tf.float64, shape = [None, 10], name = 'observation')


In [ ]:
# layer 1 - 100 neurons
w1 = tf.Variable(tf.truncated_normal([28 * 28, 100], stddev=1.0/math.sqrt(float(28*28)), dtype=tf.float64))
b1 = tf.Variable(tf.zeros([100], dtype = tf.float64))
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

# layer 2 - 50 neurones
w2 = tf.Variable(tf.truncated_normal([100, 50], stddev=1.0/math.sqrt(float(100)), dtype=tf.float64))
b2 = tf.Variable(tf.zeros([50], dtype = tf.float64))
l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)

# layer 3 - 10 neurons. softmax / output layer
w3 = tf.Variable(tf.truncated_normal([50, 10], stddev=1.0/math.sqrt(float(50)), dtype = tf.float64))
b3 = tf.Variable(tf.zeros([10], dtype = tf.float64))
y = tf.nn.softmax(tf.nn.sigmoid(tf.matmul(l2, w3) + b3))
#y = tf.matmul(l2, w3) + b3

In [ ]:
# Error
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=1))
#cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, y_)
#loss = tf.reduce_mean(cross_entropy, reduction_indices=1)

In [ ]:
print(y)

In [ ]:
print(y_)

In [ ]:
# Training Setting
#train_step = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

## Loading input data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})
    #sess.run([train_step], feed_dict={x:batch[0], y_:batch[1]})


In [ ]:
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))


In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64), reduction_indices=0)


In [ ]:
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))


In [ ]:
sess.close()

----

## 연습장

In [ ]:
test_x = [[1, 1, 1], [1, 1, 1]]
print(tf.reduce_sum(test_x))
print(test_x)
print(tf.reduce_sum(test_x, 1))

In [ ]:
# tensor와 value
sess = tf.InteractiveSession()
sum_val = sess.run(tf.reduce_sum(test_x, 0))
sess.close()
print(sum_val)
print(tf.reduce_sum(test_x, 0))

In [ ]:
print(w[:,0])
print(w[0, :])
print(w[0, 0])

In [ ]:
print(tf.zeros([784, 10]))

In [ ]:
import numpy as np

In [ ]:
np.zeros([784, 10])

In [ ]:
# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
#
# The value returned by the constructor represents the output
# of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix
# multiplication.
product = tf.matmul(matrix1, matrix2)

In [ ]:
print(product)
sess = tf.Session()
result = sess.run(product)
print(result)
sess.close()

In [ ]:
with tf.Session() as s:
    result = s.run(product)
    print(result)

In [ ]:
# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
  # Run the 'init' op
  sess.run(init_op)
  # Print the initial value of 'state'
  print(sess.run(state))
  # Run the op that updates 'state' and print 'state'.
  for _ in range(3):
    sess.run(update)
    print(sess.run(state))


In [ ]:
input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print(result)

# output:
# [array([ 21.], dtype=float32), array([ 7.], dtype=float32)]